In [1]:
import os
import pandas as pd
import mysql.connector
from mysql.connector import Error
import datetime
import shutil

# 데이터베이스 설정
db_config = {
    "host": "localhost",
    "database": "teckwah_test",
    "user": "root",
    "password": "1234",
    "allow_local_infile": True,
}

# 다운로드 폴더 경로 설정
download_folder = "C:\\MyMain\\Teckwah\\download\\xlsx_files"
xlsx_complete_folder = os.path.join("C:\\MyMain\\Teckwah\\download\\", "xlsx_files_complete")

# 폴더가 존재하지 않으면 생성
os.makedirs(xlsx_complete_folder, exist_ok=True)

def get_latest_file(download_folder):
    """최신 다운로드된 파일을 찾습니다."""
    files = [os.path.join(download_folder, f) for f in os.listdir(download_folder) if f.endswith(".xlsx")]
    latest_file = max(files, key=os.path.getctime)
    return latest_file

def get_dell_week_and_fy(date_str):
    """날짜를 Dell-Week와 Dell_FY로 변환합니다."""
    date = datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    fy_start_month = 2
    fy_start_day = 1

    if date.month < fy_start_month or (date.month == fy_start_month and date.day < fy_start_day):
        fy = date.year - 1
    else:
        fy = date.year

    fy_start_date = datetime.datetime(fy, fy_start_month, fy_start_day)
    delta = date - fy_start_date
    dell_week = (delta.days // 7) + 1
    dell_week_str = f"WK{str(dell_week).zfill(2)}"
    fy_str = f"FY{str(fy % 100).zfill(2)}"

    return dell_week_str, fy_str

def get_quarter(date_str):
    """날짜를 분기로 변환합니다."""
    date = datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    return f"Q{((date.month - 1) // 3) + 1}"

def get_order_type(edi_order_type):
    """EDI_Order_Type을 OrderType으로 변환합니다."""
    order_type_mapping = {
        "BALANCE-IN": "P3",
        "REPLEN-IN": "P3",
        "PNAE-IN": "P1",
        "PNAC-IN": "P1",
        "DISPOSE-IN": "P6",
        "PURGE-IN": "P6",
    }
    return order_type_mapping.get(edi_order_type, "Unknown")

# 최신 엑셀 파일 가져오기
excel_file_path = get_latest_file(download_folder)

# 엑셀 파일 읽기
df = pd.read_excel(excel_file_path)

# 필요한 데이터 변환
df['PutAwayDate'] = pd.to_datetime(df['PutAwayDate'], format='%Y-%m-%d %H:%M:%S')
df['Dell_Week'], df['Dell_FY'] = zip(*df['PutAwayDate'].apply(lambda x: get_dell_week_and_fy(x.strftime('%Y-%m-%d %H:%M:%S'))))
df['Quarter'] = df['PutAwayDate'].apply(lambda x: get_quarter(x.strftime('%Y-%m-%d %H:%M:%S')))
df['OrderType'] = df['EDI_Order_Type'].apply(get_order_type)
df['Month'] = df['PutAwayDate'].dt.strftime('%B')

# Count_RC 및 Count_PO 계산 (추가 설명 필요)
# 현재 엑셀 데이터에서 해당 값 가져오기
# 예시: df['Count_RC'] = df['Count_RC_Column']
# 예시: df['Count_PO'] = df['Count_PO_Column']

# 데이터베이스에 연결
try:
    connection = mysql.connector.connect(
        host=db_config["host"],
        database=db_config["database"],
        user=db_config["user"],
        password=db_config["password"],
        allow_local_infile=db_config["allow_local_infile"]
    )

    if connection.is_connected():
        cursor = connection.cursor()

        # OrderType 테이블 생성
        cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS OrderType (
                EDI_Order_Type VARCHAR(255) PRIMARY KEY,
                Detailed_Order_Type VARCHAR(255)
            )
            """
        )

        # Receiving_TAT_Report 테이블 생성
        cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS Receiving_TAT_Report (
                ReceiptNo VARCHAR(255) PRIMARY KEY,
                Customer_Order_No VARCHAR(255),
                PO_No VARCHAR(255),
                Part VARCHAR(255),
                EDI_Order_Type VARCHAR(255),
                Ship_From VARCHAR(255),
                Ship_to VARCHAR(255),
                Country VARCHAR(255),
                Quantity BIGINT(45),
                PutAwayDate DATETIME,
                Dell_FY VARCHAR(20),
                Quarter VARCHAR(10),
                Month VARCHAR(20),
                Dell_Week VARCHAR(10),
                OrderType VARCHAR(255),
                Count_RC INT,
                Count_PO INT,
                FOREIGN KEY (EDI_Order_Type) REFERENCES OrderType(EDI_Order_Type)
            )
            """
        )

        # 데이터 삽입을 위한 준비
        for i, row in df.iterrows():
            # OrderType 테이블에 데이터 삽입
            ordertype_sql = """
            INSERT IGNORE INTO OrderType (EDI_Order_Type, Detailed_Order_Type) VALUES (%s, %s)
            """
            cursor.execute(ordertype_sql, (row['EDI_Order_Type'], row['Detailed_Order_Type']))

            # Receiving_TAT_Report 테이블에 데이터 삽입
            receiving_tat_sql = """
            INSERT INTO Receiving_TAT_Report (
                ReceiptNo, Customer_Order_No, PO_No, Part, EDI_Order_Type, Ship_From, Ship_to, Country,
                Quantity, PutAwayDate, Dell_FY, Quarter, Month, Dell_Week, OrderType, Count_RC, Count_PO
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(receiving_tat_sql, (
                row['ReceiptNo'], row['Customer_Order_No'], row['PO_No'], row['Part'], row['EDI_Order_Type'],
                row['Ship_From'], row['Ship_to'], row['Country'], row['Quantity'], row['PutAwayDate'], row['Dell_FY'],
                row['Quarter'], row['Month'], row['Dell_Week'], row['OrderType'], row['Count_RC'], row['Count_PO']
            ))

        # 데이터베이스에 커밋
        connection.commit()

        # 엑셀 파일을 완료된 폴더로 이동
        shutil.move(excel_file_path, os.path.join(xlsx_complete_folder, os.path.basename(excel_file_path)))

except Error as e:
    print("Error while connecting to MySQL", e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")


ValueError: time data "05/28/2024 12:54:03" doesn't match format "%Y-%m-%d %H:%M:%S", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [2]:
import pandas as pd

# 파일 경로 설정

# 엑셀 파일 읽기
df = pd.read_excel(file_path, sheet_name='CS Receiving TAT')

# Quantity 컬럼 내용 확인
quantity_values = df['Quantity'].head(20)
print('Initial Quantity values:', quantity_values)

# Quantity 컬럼을 정수로 변환 (변환되지 않는 값은 NaN으로 설정)
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')

# 변환되지 않은 값들 확인
invalid_values = df[df['Quantity'].isna()]['Quantity']
print('Invalid Quantity values:', invalid_values)

# NaN 값을 0으로 대체
df['Quantity'] = df['Quantity'].fillna(0).astype(int)

# 변환 후 데이터 확인
print('Converted Quantity values:', df['Quantity'].head(20))


Initial Quantity values: 0     1
1     1
2     1
3     1
4     1
5     3
6     1
7     4
8     1
9     1
10    2
11    1
12    1
13    3
14    1
15    2
16    1
17    1
18    1
19    2
Name: Quantity, dtype: int64
Invalid Quantity values: Series([], Name: Quantity, dtype: int64)
Converted Quantity values: 0     1
1     1
2     1
3     1
4     1
5     3
6     1
7     4
8     1
9     1
10    2
11    1
12    1
13    3
14    1
15    2
16    1
17    1
18    1
19    2
Name: Quantity, dtype: int32
